# Limpieza de datos con PySpark: Data Science Job Posting on Glassdoor

Los [datos](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ETYTQ0c-i6FLjM8rZ4iT1cgB6ipFAkainM-4V9M8DXsBiA?e=PeMtvh) fueron extraídos (scrapeados) del sitio web de Glassdoor y recoge los salarios de distintos puestos relacionados a Data.

### Resolver los siguientes requerimientos, para cada operación/moficación imprima como van quedando los cambios.

1. Cargar los datos y mostrar el esquema o la informacion de las columnas y el tip de dato de cada columna

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Data_Science_Glassdoor").getOrCreate()
 
# Leer el archivo Excel usando PySpark
df = spark.read.option("header", "true") \
               .option("delimiter", ";") \
               .option("multiline", "true") \
               .option("quote", "\"") \
               .option("escape", "\"") \
               .option("inferSchema", "true") \
               .csv("/FileStore/PySpark-Data-Cleansing/Caso_4/ds_jobs.csv")
 
# Mostrar el esquema y las filas del DataFrame
df.printSchema()
df.display()

2. Eliminar duplicados

In [0]:
import pandas as pd

# Crear un DF de Pandas a partir del DF de PySpark
df_pd = df.toPandas()

# Eliminar el índice para evitar que PySpark crea que todas las filas, incluso las duplicadas, son únicas
df_pd.drop('index', axis=1, inplace=True)

# Ver, respectivamente, el número de filas y de columnas del DF
print(df_pd.shape)

In [0]:
# Ver el número de filas con duplicados
print(df_pd[df_pd.duplicated()].shape)

# Mostrar las filas que son
df_pd[df_pd.duplicated()]

In [0]:
# Eliminar duplicados
df_pd.drop_duplicates(inplace=True)

df_pd.shape

3. Decidir que hacer con los datos faltantes 

In [0]:
# Rellenar con '0' los datos faltantes
df_pd.fillna(0, inplace=True)

4. Decidir que hacer con los valores nulos

In [0]:
# Eliminar los valores nulos
df_pd.dropna(inplace=True)

5. ¿Cuántos registros tiene el csv?

In [0]:
df_pd.shape

6. Mostrar los valores únicos de `Job title` 

In [0]:
print(f"Número de valores únicos de 'Job title': {df_pd['Job Title'].nunique()}")
print(df_pd['Job Title'].unique())

7. Remover la letra `K` de la columna `Salary Estimate` y multiplicar por 1000.

In [0]:
# Eliminar "$" y "," y separar el rango de salarios
df_pd['Salary Estimate'] = df_pd['Salary Estimate'].str.replace('[$,]', '', regex=True).str.replace('[K,]', '', regex=True)

# Extraer el valor mínimo (antes del guion) y convertir a flotante, luego multiplicar por 1000
df_pd['Salary Estimate'] = df_pd['Salary Estimate'].str.split('-').str[0].astype(float) * 1000

# Mostrar el DataFrame actualizado
df_pd.head()

8. Mostrar los valores únicos del campo `Salary Estimate`

In [0]:
print(f"Número de valores únicos de 'Salary Estimate': {df_pd['Salary Estimate'].nunique()}")
print(df_pd['Salary Estimate'].unique())

9. Eliminar `(Glassdoor est.)` y `(Employer est.)` del campo `Salary Estimate`

Ya fue realizado en el paso 7.

10. Mostrar de mayor a menor los valores del campo `Salary Estimate`

In [0]:
# Ordenar el DataFrame de mayor a menor en función de 'Salary Estimate'
df_pd = df_pd.sort_values(by='Salary Estimate', ascending=False)

df_pd.head()

11. De la columna `Job Description` quitar los saltos de linea `\n` del texto

In [0]:
df_pd['Job Description'] = df_pd['Job Description'].str.replace(r'\n', ' ', regex=True)

# Verificar el resultado
print(df_pd['Job Description'].head())

12. De la columna `Rating` muestre los valores unicos.

In [0]:
print(f"Número de valores únicos de 'Rating': {df_pd['Rating'].nunique()}")
print(df_pd['Rating'].unique())

13. Del campo `Rating` reemplazar los `-1.0` por `0.0`.

In [0]:
df_pd['Rating'] = df_pd['Rating'].replace(-1.0, 0.0)

print(df_pd['Rating'].head(10))

14. Mostrar los valores unicos y ordenar los valores del campo `Company Name`.

In [0]:
print("Valores únicos de 'Company Name':")
print(df_pd['Company Name'].unique())

# Ordenar alfabéticamente los valores de 'Company Name'
df_pd_sorted_company = df_pd.sort_values(by='Company Name')

# Mostrar los primeros 10 valores ordenados
print("\nPrimeros 10 valores ordenados de 'Company Name':")
print(df_pd_sorted_company[['Company Name']].head(10))

15. Quitar todos los caracteres innecesarios que encuentres en el campo `Company Name`. Por ejemplo los saltos de linea `\n`

In [0]:
df_pd['Company Name'] = df_pd['Company Name'].str.replace(r'\n', ' ', regex=True)

# Mostrar los primeros 10 valores de 'Company Name' después de la limpieza
print("\nPrimeros 10 valores de 'Company Name' después de la limpieza:")
print(df_pd[['Company Name']].head(10))

16. En el campo `Location` convertir esa columna en dos: `City` y `State`. Las ciudades que tengas en `Location` asignar a la columna `City`. Lo mismo para `State`. Luego elimine la columna `Location`.

In [0]:
# Dividir la columna Location, llenando valores faltantes con NaN
split_location = df_pd['Location'].str.split(',', expand=True)

# Asignar las columnas City y State
df_pd['City'] = split_location[0].str.strip()  # Primera parte (City)
df_pd['State'] = split_location[1].str.strip() if 1 in split_location.columns else None  # Segunda parte (State)

# Eliminar la columna original Location
df_pd.drop(columns=['Location'], inplace=True)

# Verificar el resultado
print(df_pd[['City', 'State']].head())

17. Repetir la misma lógica de la pregunta 16 pero para el campo `Headquarters`. En Headquarters dejar solo la ciudad, mientras que para el estado añadirla a una columna nueva ` Headquarter State`.

In [0]:
# Dividir la columna Headquarters en dos partes: Headquarters City y State
split_headquarters = df_pd['Headquarters'].str.split(',', expand=True)

# Mantener solo la ciudad en la columna Headquarters
df_pd['Headquarters'] = split_headquarters[0].str.strip()  # Primera parte (City)

# Crear una columna adicional para el estado si lo deseas
df_pd['Headquarter State'] = split_headquarters[1].str.strip() if 1 in split_headquarters.columns else None

# Verificar el resultado
print(df_pd[['Headquarters', 'Headquarter State']].head())

18. Muestre los valores únicos del campo `Headquarter State` 

In [0]:
print(f"Número de valores únicos de 'Headquarter State': {df_pd['Headquarter State'].nunique()}")
print(df_pd['Headquarter State'].unique())

19. Mostrar valores unicos del campo `Size`.

In [0]:
print(f"Número de valores únicos de 'Size': {df_pd['Size'].nunique()}")
print(df_pd['Size'].unique())

20. Quitar 'employee' de los registros del campo `Size`. Elimine tambien otros caracteres basura.

In [0]:
df_pd['Size'] = df_pd['Size'].str.replace(r'\s?employees', '', regex=True)  # Elimina la palabra 'employees'
df_pd['Size'] = df_pd['Size'].str.replace(r'\s?[^a-zA-Z0-9\s]', '', regex=True)  # Elimina caracteres no alfanuméricos

# Verificar el resultado
print(df_pd[['Size']].head())

21. Reemplazar la palabra 'to' por '-' en todos los registros del campo `Size`. Reemplazar tambien '-1' por 'Unknown'. 

In [0]:
df_pd['Size'] = df_pd['Size'].str.replace('to', '-', regex=False)  # Reemplaza 'to' por '-'
df_pd['Size'] = df_pd['Size'].str.replace('-1', 'Unknown', regex=False)  # Reemplaza '-1' por 'Unknown'

# Verificar el resultado
print(df_pd[['Size']].head(10))

22. Mostrar el tipo de dato del campo `Type of ownership` y sus registros unicos.

In [0]:
# Ver el tipo de dato del campo 'Type of ownership'
print(f"Tipo de dato del campo 'Type of ownership': {df_pd['Type of ownership'].dtype}")

print(f"Número de valores únicos de 'Type of ownership': {df_pd['Type of ownership'].nunique()}")
print(df_pd['Type of ownership'].unique())

23. Cambiar '-1' por 'Unknown' en todos los registros del campo `Type of ownership`.

In [0]:
df_pd['Type of ownership'] = df_pd['Type of ownership'].replace('-1', 'Unknown')

# Verificar el resultado
print(df_pd[['Type of ownership']].head(20))

24. Cambiar:  
-  `Company - Public` por `Public Company`  
-  `Company - Private` por `Private Company`  
-  `Private Practice / Firm` por `Private Company`  
-  `Subsidiary or Business Segment` por `Business`  
-  `College / University` por `Education`  
En todos los registros del campo `Type of ownership`.

In [0]:
df_pd['Type of ownership'] = df_pd['Type of ownership'].replace({
    'Company - Public': 'Public Company',
    'Company - Private': 'Private Company',
    'Private Practice / Firm': 'Private Company',
    'Subsidiary or Business Segment': 'Business',
    'College / University': 'Education'
})

# Verificar el resultado
print(df_pd[['Type of ownership']].head(20))

25. Mostrar el tipo de dato y los valores unicos del campo `Industry`.

In [0]:
print(f"Tipo de dato del campo 'Industry': {df_pd['Industry'].dtype}")
print(f"Número de valores únicos de 'Industry': {df_pd['Industry'].nunique()}")
print(df_pd['Industry'].unique())

26. En el mismo campo de `Industry` reemplazar '-1' por 'Not Available' y '&' por 'and'.  Vuelva a imprimir los valores unicos en orden alfabético.

In [0]:
# Reemplazar '-1' por 'Not Available' y '&' por 'and' en la columna Industry
df_pd['Industry'] = df_pd['Industry'].replace({'-1': 'Not Available', '&': 'and'})

# Mostrar los valores únicos de la columna Industry en orden alfabético
print(sorted(df_pd['Industry'].unique()))

27. Para el campo `Sector`, muestre el tipo de dato y los valores únicos.

In [0]:
print(f"Tipo de dato del campo 'Sector': {df_pd['Sector'].dtype}")
print(f"Número de valores únicos de 'Sector': {df_pd['Sector'].nunique()}")
print(df_pd['Sector'].unique())

28. Aplica la misma lógica de la pregunta 26 pero sobre el campo `Sector`.

In [0]:
# Reemplazar '-1' por 'Not Available' y '&' por 'and' en la columna Sector
df_pd['Sector'] = df_pd['Sector'].replace({'-1': 'Not Available', '&': 'and'})

# Mostrar los valores únicos de la columna Sector en orden alfabético
print(sorted(df_pd['Sector'].unique()))

29. Para el campo `Revenue`, muestre el tipo de dato y los valores únicos en orden ascedente.

In [0]:
print(f"Tipo de dato del campo 'Revenue': {df_pd['Revenue'].dtype}")
print(f"Número de valores únicos de 'Revenue': {df_pd['Revenue'].nunique()}")
print(sorted(df_pd['Revenue'].unique()))

30. En el campo `Revenue`, cambiar:  
-  `-1` por `N/A`  
-  `Unknown / Non-Applicable` por `N/A`  
-  `Less than $1 million (USD)` por `Less than 1`
-  Quitar `$` y `(USD)`

In [0]:
# Reemplazar los valores específicos en la columna
df_pd['Revenue'] = df_pd['Revenue'].replace({
    '-1': 'N/A',
    'Unknown / Non-Applicable': 'N/A',
    'Less than $1 million (USD)': 'Less than 1'
})

# Quitar el símbolo '$' y la cadena '(USD)'
df_pd['Revenue'] = df_pd['Revenue'].str.replace('$', '', regex=False)
df_pd['Revenue'] = df_pd['Revenue'].str.replace('(USD)', '', regex=False)

# Verificar el resultado
print(df_pd[['Revenue']].head(20))

31. Borrar el campo `Competitors`.

In [0]:
df_pd = df_pd.drop(columns=['Competitors'])
df_pd.head()

32. Crear tres columnas: `min_salary` (salario mínimo), `max_salary` (salario maximo) y `avg_salary` (salario promedio) a partir de los datos del campo `Salary Estimate`.

In [0]:
df_pd['min_salary'] = df_pd['Salary Estimate']  # Mínimo
df_pd['max_salary'] = df_pd['Salary Estimate']  # Máximo
df_pd['avg_salary'] = df_pd['Salary Estimate']  # Promedio

# Verificar el resultado
print(df_pd[['Salary Estimate', 'min_salary', 'max_salary', 'avg_salary']].head(30))

33. Mostrar los valores unicos del campo `Founded` y el tipo de dato.

In [0]:
print(f"Tipo de dato del campo 'Founded': {df_pd['Founded'].dtype}")
print(f"Número de valores únicos de 'Founded': {df_pd['Founded'].nunique()}")
print(sorted(df_pd['Founded'].unique()))

34. Reemplazar '-1' por '2024' en todos los registros del campo `Founded`.

In [0]:
df_pd['Founded'] = df_pd['Founded'].replace(-1, 2024)

# Verificar el resultado
print(df_pd[['Founded']].head(10))

35. Crear una nueva columna o campo que se llame `company_age` con los datos que se deducen del campo `Founded`.

In [0]:
from datetime import datetime

# Obtener el año actual
current_year = datetime.now().year

# Crear la nueva columna 'company_age' restando el año de 'Founded' al año actual
df_pd['company_age'] = current_year - df_pd['Founded']

# Verificar la creación de la nueva columna
print(df_pd[['Founded', 'company_age']].head())

36. Crear una columna o campo que se llame: `Job Type` y en cada registro debe ir Senior, Junior o NA según los datos del campo `Job Title`.  
- Cambiar 'sr' o 'senior' o 'lead' o 'principal' por `Senior` en el campo `Job Type`. No olvidar las mayúsculas.
- Cambiar 'jr' o 'jr.' o cualquier otra variante por `Junior`.  
- En cualquier otro caso distinto a los anteriores añadir NA.

In [0]:
# Crear la columna Job Type basada en palabras clave en Job Title
df_pd['Job Type'] = df_pd['Job Title'].apply(
    lambda x: 'Senior' if pd.notnull(x) and any(keyword in x.lower() for keyword in ['senior', 'sr', 'lead', 'principal'])
    else 'Junior' if pd.notnull(x) and any(keyword in x.lower() for keyword in ['junior', 'jr', 'jr.'])
    else 'NA'
)

# Verificar el resultado
print(df_pd[['Job Title', 'Job Type']].head(20))

37. Muestra los registros únicos del campo `Job Type`. 

In [0]:
print(f"Tipo de dato del campo 'Job Type': {df_pd['Job Type'].dtype}")
print(f"Número de valores únicos de 'Job Type': {df_pd['Job Type'].nunique()}")
print(sorted(df_pd['Job Type'].unique()))

38. Partiendo del campo `Job Description` se extraer todas o las principales skills solicitadas por las empresas, por ejemplo: Python, Spark , Big Data. Cada Skill debe ir en una nueva columna de tipo Binaria ( 0 , 1) o Booleana (True,  False) de modo que cada skill va ser una nueva columna y si esa skill es solicitada por la empresa colocar 1 sino colocar 0. Por ejemplo:  

Por ejemplo:  
| Job Title         | Salary Estimate | Job Description                                 | Rating | Company Name       | Size       | Founded | Type of ownership         | Industry                       | Sector                         | Same State      | company_age | Python | Excel |
|--------------------|-----------------|-------------------------------------------------|--------|--------------------|------------|---------|---------------------------|--------------------------------|--------------------------------|----------------|-------------|--------|-------|
| Sr Data Scientist | 137000-171000   | Description The Senior Data Scientist is resp... | 3.1    | Healthfirst        | 1001-5000  | 1993    | Nonprofit Organization    | Insurance Carriers            | Insurance Carriers            | Same State      | 31          | 0      | 0     |
| Data Scientist    | 137000-171000   | Secure our Nation, Ignite your Future Join th... | 4.2    | ManTech            | 5001-10000 | 1968    | Public Company            | Research and Development      | Research and Development      | Same State      | 56          | 0      | 0     |
| Data Scientist    | 137000-171000   | Overview Analysis Group is one of the larges... | 3.8    | Analysis Group      | 1001-5000  | 1981    | Private Company           | Consulting                    | Consulting                    | Same State      | 43          | 1      | 1     |
| Data Scientist    | 137000-171000   | JOB DESCRIPTION: Do you have a passion for Da... | 3.5    | INFICON            | 501-1000   | 2000    | Public Company            | Electrical and Electronic Manufacturing | Electrical and Electronic Manufacturing | Different State | 24          | 1      | 1     |


In [0]:
# Lista de skills a buscar en Job Description
skills = ['python', 'spark', 'big data', 'java', 'machine learning', 'data analysis', 'hadoop', 'sql', 'r', 'tensorflow', 'aws', 'azure']

# Crear una nueva columna binaria para cada skill
for skill in skills:
    df_pd[skill] = df_pd['Job Description'].str.contains(skill, case=False, na=False).astype(int)

# Verificar el resultado mostrando las primeras filas de las nuevas columnas
print(df_pd[skills + ['Job Description']].head(10))

39. Exportar dataset final a csv

In [0]:
df_spark = spark.createDataFrame(df_pd)
df_spark.printSchema()
df_spark.display()

40. Extraer todos los insights posibles que sean de valor o utilidad. Cree nuevas columnas, agrupar,  filtrar hacer varios plots que muestren dichos insights que sean de utilidad para una empresa o para un usuario. Elabore conclusiones con los insights encontrados. 

**Distribución de la edad en las empresas**: Este gráfico muestra cómo se distribuyen las edades de las empresas. Si la mayoría de las empresas tienen una edad baja, puede indicar que las empresas nuevas son más comunes en el dataset, lo cual podría reflejar un mercado en crecimiento o dinámico. Las empresas más antiguas podrían ser menos numerosas.

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.hist(df_pd['company_age'].dropna(), bins=20, color='salmon', edgecolor='black')
plt.title('Distribución de la Edad de las Empresas')
plt.xlabel('Edad de la Empresa')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

**Número de empresas por sector**: Este gráfico muestra cuántas empresas están presentes en cada sector. Un número alto de empresas en ciertos sectores puede indicar una alta demanda de trabajos en esas áreas, mientras que un número bajo en otros sectores puede reflejar una menor representación de esas industrias. Sectores como tecnología o finanzas suelen tener más ofertas de empleo, mientras que sectores menos populares podrían estar menos representados.

In [0]:
import seaborn as sns

# Contar el número de empresas por sector
company_count_by_sector = df_pd['Sector'].value_counts()

# Gráfico de barras de empresas por sector
plt.figure(figsize=(10, 6))
sns.barplot(x=company_count_by_sector.index, y=company_count_by_sector.values, palette='viridis')
plt.title('Número de Empresas por Sector')
plt.xlabel('Sector')
plt.ylabel('Número de Empresas')
plt.xticks(rotation=45, ha='right')
plt.show()

**Salario promedio por estado**: Los estados con mayores salarios promedio podrían indicar una alta concentración de empresas tecnológicas o industrias bien remuneradas. Estos estados podrían destacarse debido a la presencia de grandes empresas de tecnología y startups.

In [0]:
# Calcular el salario promedio por estado
salary_by_state = df_pd.groupby('State')['avg_salary'].mean().sort_values(ascending=False)

# Gráfico de barras del salario promedio por estado
plt.figure(figsize=(12, 8))
salary_by_state.plot(kind='bar', color='skyblue')
plt.title('Salario promedio por estado')
plt.xlabel('Estado')
plt.ylabel('Salario Promedio')
plt.xticks(rotation=90)
plt.show()

**Relación entre la antigüedad de la empresa y el salario promedio**: 

In [0]:
# Gráfico de dispersión entre la antigüedad de la empresa y el salario promedio
plt.figure(figsize=(10, 6))
plt.scatter(df_pd['company_age'], df_pd['avg_salary'], alpha=0.5, color='purple')
plt.title('Relación entre la antigüedad de la empresa y el salario promedio')
plt.xlabel('Antigüedad de la empresa (Años)')
plt.ylabel('Salario promedio')
plt.show()

**Habilidades más solicitadas**: Un análisis sobre cuántas veces se menciona cada habilidad en la descripción del trabajo, reflejando las tendencias acuales del sector.

In [0]:
# Contar la cantidad de veces que cada habilidad es mencionada
skills_columns = ['python', 'spark', 'big data', 'java', 'machine learning', 'data analysis', 'hadoop', 'sql', 'r', 'tensorflow', 'aws', 'azure']
skills_count = df_pd[skills_columns].sum().sort_values(ascending=False)

# Gráfico de barras de las habilidades más solicitadas
plt.figure(figsize=(12, 8))
skills_count.plot(kind='bar', color='lightcoral')
plt.title('Habilidades más solicitadas')
plt.xlabel('Habilidad')
plt.ylabel('Cantidad de veces mencionada')
plt.xticks(rotation=45)
plt.show()

**Distribución de salarios por tipo de empleo**: Las posiciones Senior pueden mostrar una distribución de salarios más alta que las Junior, lo que refleja la mayor experiencia y responsabilidad solicitada para esos roles.

In [0]:
# Crear un gráfico de caja para ver la distribución de los salarios por tipo de empleo
plt.figure(figsize=(10, 6))
df_pd.boxplot(column='avg_salary', by='Job Type', patch_artist=True, medianprops=dict(color='black'))
plt.title('Distribución de salario por tipo de empleo')
plt.suptitle('')
plt.xlabel('Tipo de empleo')
plt.ylabel('Salario promedio')
plt.show()

**Distribución de salarios por sector**: Ver si ciertos sectores (por ejemplo, tecnología, finanzas) tienden a pagar más.

In [0]:
# Calcular el salario promedio por sector
salary_by_sector = df_pd.groupby('Sector')['avg_salary'].mean().sort_values(ascending=False)

# Gráfico de barras del salario promedio por sector
plt.figure(figsize=(12, 8))
salary_by_sector.plot(kind='bar', color='lightgreen')
plt.title('Salario promedio por sector')
plt.xlabel('Sector')
plt.ylabel('Salario promedio')
plt.xticks(rotation=90)
plt.show()

**Análisis del salario promedio por industria**: Ver qué industrias ofrecen los salarios más altos.

In [0]:
# Calcular el salario promedio por industria
salary_by_industry = df_pd.groupby('Industry')['avg_salary'].mean().sort_values(ascending=False)

# Gráfico de barras del salario promedio por industria
plt.figure(figsize=(12, 8))
salary_by_industry.plot(kind='bar', color='lightblue')
plt.title('Salario promedio por industria')
plt.xlabel('Industria')
plt.ylabel('Salario promedio')
plt.xticks(rotation=90)
plt.show()